# This notebook accomplishes the following:
1. Computes consensus signatures for LINCS compounds associated with CNS,CV, and blood disease areas (based on efforts from Leadscope/IDG) and produces up/down gene sets for each compound.
<br><br>
2. Produces Knowledge Graph ready outputs for chemical-gene associations from LINCS, chemical-disease area associations from IDG/Leadscope, and Cardiovascular defect-gene associations from SPARC/KidsFirst.

## 1. Generating gene sets from LINCS chemical perturbagen signatures that are associated with Blood, CNS, and CV Disease Areas

### Data sources:
    
Leadscope/IDG compounds mapped to LINCS compounds with associated disease area: https://docs.google.com/spreadsheets/d/1ZCPnn47B2QTfiSpzNCJh86m7SAh9GG2LFaAk1xdfSSg/edit

Gene-Cardiovascular system defects associations: https://github.com/nih-cfde/ReproToxTables/blob/main/Great_Vessel_Associated_Diseases_v1.xlsx

L1000FWD metadata: https://maayanlab.cloud/L1000FWD/download/Drugs_metadata.csv

LINCS 2021 chemical perturbagen metadata: https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/gctx/metadata/cp_siginfo_abr.txt

LINCS 2021 chemical metadata: https://s3.amazonaws.com/lincs-dcic/sigcom-lincs-metadata/LINCS_small_molecules.tsv

LINCS 2021 chemical perturbagen gene expression matrix: https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/gctx/cd-coefficient/cp_coeff_mat.gctx

In [ ]:
import os

import pandas as pd
import numpy as np
import h5py as h5

import requests
from tqdm import tqdm
import time
import json

from rdflib import Graph, RDF, Namespace

In [ ]:
def matrix_slice(query_name):
    '''
    Get slice of complete signature matrix by drug name
    'pert_names'(list), 'f'(h5 object),and 'genes'(list) variables are defined outside the scope of the function.
    '''
    col_idx = [i for i, x in enumerate(pert_names) if x == query_name]
    values = f['0']['DATA']['0']['matrix'][col_idx,:]
    return pd.DataFrame(values.T, columns=[query_name]*len(col_idx), index=genes, dtype=float)

In [ ]:
# Leadscope compounds with annotated disease areas that overlap with LINCS chemical library
reprotox_drugs = pd.read_excel('input/ReproTox_export.xlsx',
                              names = ['Canonical_SMILES','LS_id','DiseaseArea','LSM_id'])

# L1000FWD metadata for small molecules (which contains LSM IDs to map to Leadscope compounds)
l1000fwd_meta = pd.read_csv('input/Drugs_metadata.csv')

# LINCS 2021 chemical perturbagen signature metadata
drug_meta = pd.read_table('https://lincs-dcic.s3.amazonaws.com/LINCS-sigs-2021/gctx/metadata/cp_siginfo_abr.txt')

# Table of BRD-IDs mapped to aliases not included in the original LINCS 2021 metadata
drug_alias_table = pd.read_csv('https://s3.amazonaws.com/lincs-dcic/sigcom-lincs-metadata/LINCS_small_molecules.tsv',
                              sep = '\t',
                              index_col=0)

alias_lookup =  drug_alias_table[~(drug_alias_table['compound_aliases'] == '-')]\
                .to_dict()['compound_aliases']

In [ ]:
f = h5.File('input/cp_coeff_mat.gctx', 'r')
genes = [x.decode('UTF-8') for x in f['0']['META']['ROW']['id']]

In [ ]:
# Map BRD-IDs without drug name to drug name (if applicable)
# This is a list of all small molecule perturbagens in the LINCS gene expression matrix
# These have to be in the order given by source metadata to match matrix indices when slicing
pert_names = [alias_lookup.get(x,x) for x in drug_meta['pert_name'].tolist()]

In [ ]:
# Merge L1000FWD metadata with Leadscope compound table on common LSM Ids
merged_table = l1000fwd_meta.merge(reprotox_drugs, on = 'LSM_id')[['pert_id','LSM_id','pert_iname','DiseaseArea']]
merged_table = merged_table[merged_table['pert_iname'].isin(set(pert_names))].drop_duplicates().reset_index(drop=True)
merged_table.to_csv('ReproTox_Leadscope_IDG_LINCS_compound_table.csv', index = False)

In [ ]:
merged_table.head()

In [ ]:
# Set of compounds from Leadscope matched to LINCS small molecules by LSM ID
drug_names = set(merged_table['pert_iname'].tolist())

In [ ]:
# Iterate through all signatures for a particular chemical and determine 'consensus signature' based on
# largest absolute Characteristic Direction coefficient mean value
sigs = []
correlation_data = []
for drug in tqdm(drug_names):
    drug_matrix = matrix_slice(drug)
    if len(drug_matrix.columns) > 1:
        correlation_score = 0.0
        for i,x in enumerate(drug_matrix.columns):
            current_signature = drug_matrix.iloc[:,i]
            current_score = np.absolute(current_signature.values).mean()
            if current_score > correlation_score:
                correlation_score = current_score
                consensus_signature = pd.DataFrame(current_signature)
        sigs.append(consensus_signature)
        correlation_data.append([drug,correlation_score,len(drug_matrix.columns)])
        
    else:
        sigs.append(drug_matrix)

In [ ]:
# Concatenate all consensus signatures into one dataframe
consensus_mat = pd.concat(sigs,axis=1)

# Drop NaN columns
consensus_mat.dropna(axis = 1,inplace=True)

In [ ]:
consensus_mat.head()

In [ ]:
# Create up/down genesets from each consensus signature for each chemical
up_genes = {}
down_genes = {}
for i,r in consensus_mat.T.iterrows():
    up_genes[i] = r.sort_values().tail(50).index.tolist()
    down_genes[i] = r.sort_values().head(50).index.tolist()
    
pd.DataFrame.from_dict(up_genes).to_csv('ReproTox_LINCS_compounds_UpGenes.csv', index = False)
pd.DataFrame.from_dict(down_genes).to_csv('ReproTox_LINCS_compounds_DownGenes.csv', index = False)

## 2. Creating triples for Knowledge Graph ingestion

### LINCS chemical-gene associations

TSV format

In [ ]:
# Create mapping between LINCS compound names and LSM IDs
compound_mapping = dict(zip(merged_table['pert_iname'],merged_table['LSM_id']))

In [ ]:
data = []
for chemical,gene_list in up_genes.items():
    chemical_URI = "https://identifiers.org/lincs.smallmolecule:"+compound_mapping[chemical]
    relationship = "upregulates"
    relationship_URI = np.nan
    for gene in gene_list:
        gene_URI = "http://identifiers.org/hgnc.symbol/"+gene
        data.append((chemical,chemical_URI,relationship,relationship_URI,gene,gene_URI))
    
for chemical,gene_list in down_genes.items():
    chemical_URI = "https://identifiers.org/lincs.smallmolecule:"+compound_mapping[chemical]
    relationship = "downregulates"
    relationship_URI = np.nan
    for gene in gene_list:
        gene_URI = "http://identifiers.org/hgnc.symbol/"+gene
        data.append((chemical,chemical_URI,relationship,relationship_URI,gene,gene_URI))
        
pd.DataFrame(data,
            columns=['Chemical','Chemical_URI','Relationship','Relationship_URI',
                     'Gene','Gene_URI']).to_csv('LINCS_chemical_gene_associations.tsv',
                                                               sep = '\t',
                                                               index = False)

TTL format

In [ ]:
# Note: because I could not find a URI that describes upregulation and downregulation of gene expression
# all of the differentially expressed genes are simply associated with each respective compound
g = Graph()

gene_symbol = Namespace("http://identifiers.org/hgnc.symbol/")
compound = Namespace("https://identifiers.org/lincs.smallmolecule:")
relationship = Namespace("https://semanticscience.org/resource/")
compound_type = Namespace("http://purl.obolibrary.org/obo/")

g.bind('gene',gene_symbol)
g.bind('compound',compound)
g.bind('relationship',relationship)
g.bind('type',compound_type)

for drug,gene_list in up_genes.items():
    for gene in gene_list:
        lsm_id = compound_mapping[drug]
        g.add((compound[lsm_id],RDF.type,compound_type['OBI_0000040']))
        g.add((compound[lsm_id],relationship['SIO_001257'],gene_symbol[gene]))
        
for drug,gene_list in down_genes.items():
    for gene in gene_list:
        lsm_id = compound_mapping[drug]
        g.add((compound[lsm_id],RDF.type,compound_type['OBI_0000040']))
        g.add((compound[lsm_id],relationship['SIO_001257'],gene_symbol[gene]))

In [ ]:
g.serialize(format="turtle",destination='LINCS_chemical_gene_associations.ttl')

### Chemical-disease area from https://docs.google.com/spreadsheets/d/1ZCPnn47B2QTfiSpzNCJh86m7SAh9GG2LFaAk1xdfSSg/edit

TSV format

In [ ]:
# Mappings between each disease area and DOID
disease_area = {
    "CNS": "DOID_863",
    "CV": "DOID_1287",
    "Blood": "DOID_74"
}

In [ ]:
data = []
for i in range(len(merged_table)):
    row = merged_table.loc[i]
    chemical = row.pert_iname
    chemical_URI = "https://identifiers.org/lincs.smallmolecule:"+row.LSM_id
    relationship = 'chemical-disease association'
    relationship_URI = "https://semanticscience.org/resource/SIO_001257"
    disease = row.DiseaseArea
    disease_URI = "http://purl.obolibrary.org/obo/"+disease_area[disease]
    data.append((chemical,chemical_URI,relationship,relationship_URI,disease,disease_URI))
    
pd.DataFrame(data,
            columns=['Chemical','Chemical_URI','Relationship','Relationship_URI',
                     'Disease_Area','Disease_Area_URI']).to_csv('LINCS_Leadscope_IDG_compound_disease_area_associations.tsv',
                                                               sep = '\t',
                                                               index = False)

TTL format

In [ ]:
g = Graph()

compound = Namespace("https://identifiers.org/lincs.smallmolecule:")
obo = Namespace("http://purl.obolibrary.org/obo/")
relationship = Namespace("https://semanticscience.org/resource/")

g.bind('compound',compound)
g.bind('relationship',relationship)
g.bind('obo',obo)

for i in range(len(merged_table)):
    row = merged_table.loc[i]
    lsm_id = row.LSM_id
    disease = disease_area[row.DiseaseArea]
    g.add((compound[lsm_id],RDF.type,obo['OBI_0000040']))
    g.add((compound[lsm_id],relationship['SIO_000993'],obo[disease]))

In [ ]:
g.serialize(format="turtle",destination='LINCS_Leadscope_IDG_compound_diseaseArea_associations.ttl')

### Disease-gene from https://github.com/nih-cfde/ReproToxTables/blob/main/Great_Vessel_Associated_Diseases_v1.xlsx

TSV format

In [ ]:
disease_genes_table = pd.read_excel('input/Great_Vessel_Associated_Diseases_v1.xlsx',
                             index_col=0,
                             header=[1,2]).reset_index(drop=True)

In [ ]:
data = []
for col in disease_genes_table.columns:
    disease = col[0]
    hp_URI = "http://purl.obolibrary.org/obo/"+ col[1].replace(":","_")
    relationship = 'gene-disease association'
    relationship_URI = "https://semanticscience.org/resource/SIO_000983"
    for gene in disease_genes_table[col].dropna().tolist():
        gene_URI = "http://identifiers.org/hgnc.symbol/" + gene
        data.append((disease,hp_URI,relationship,relationship_URI,gene,gene_URI))
        
pd.DataFrame(data,
            columns=['Disease','Disease_URI','Relationship','Relationship_URI',
                     'Gene','Gene_URI']).to_csv('KidsFirst_GreatVesselDisease_gene_associations.tsv',
                                                               sep = '\t',
                                                               index = False)

TTL format

In [ ]:
disease_genes_table = pd.read_excel('input/Great_Vessel_Associated_Diseases_v1.xlsx',
                             skiprows=2,
                             index_col=0).reset_index(drop=True)

In [ ]:
disease_genes = {}
for disease in disease_genes_table.columns:
    disease_genes[disease.replace(":","_")] = disease_genes_table[disease].dropna().tolist()

In [ ]:
g = Graph()

obo = Namespace("http://purl.obolibrary.org/obo/")
predicate = Namespace("https://semanticscience.org/resource/")
gene_symbol = Namespace("http://identifiers.org/hgnc.symbol/")

g.bind('gene',gene_symbol)
g.bind('relationship',predicate)
g.bind('obo',obo)

for disease,gene_list in disease_genes.items():
    for gene in gene_list:
        g.add((obo[disease],RDF.type,obo['DOID_4']))
        g.add((obo[disease],predicate['SIO_000983'],gene_symbol[gene]))

In [ ]:
g.serialize(format="turtle",destination='KidsFirst_GreatVesselDisease_gene_associations.ttl')